In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance

import torch
import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# GPU Check
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    print(f"🔥 GPU: {torch.cuda.get_device_name()}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected")

print("✅ Setup complete")

In [ ]:
# === CELL 2: UNLEASHED CONFIG ===
# Maximum freedom to learn!

CONFIG = {
    # ENTRY - VERY LOOSE (let it try everything!)
    'dip_buy_rsi': 55,           # Almost any RSI
    'dip_buy_drawdown': -1,      # Tiny dip triggers
    'bounce_threshold': 1,        # Small bounces count
    'momentum_threshold': 3,      # Low momentum bar
    
    # EXIT - LET WINNERS RUN
    'profit_target_1': 12,
    'profit_target_2': 25,
    'profit_target_max': 45,
    'stop_loss': -18,            # Very wide stop
    'trailing_stop': 15,
    'max_hold_days': 90,         # Long holds allowed
    
    # POSITION - AGGRESSIVE
    'max_position_pct': 30,      # Big positions
    'min_cash_reserve': 3,       # Almost all-in allowed
    'max_positions': 20,         # Many positions
    
    # LEARNING - MAXIMUM EXPLORATION
    'episodes': 1000,
    'initial_balance': 100000,
    'exploration_start': 0.70,   # 70% random at start!
    'exploration_end': 0.25,     # 25% random at end
    'buy_boost': 5.0,            # 5x buy probability
}

WATCHLIST = [
    # AI/Chips
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'QCOM', 'CRDO', 'ANET',
    # Nuclear
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ',
    # Tech
    'HOOD', 'LUNR', 'SNOW', 'NOW',
    # Quantum
    'IONQ', 'RGTI', 'QUBT',
    # Big
    'TSLA', 'META', 'GOOGL',
    # ETF
    'SPY', 'QQQ',
    # Other
    'BA', 'RIVN', 'LYFT'
]

print(f"🔥 UNLEASHED CONFIG")
print(f"   {len(WATCHLIST)} tickers")
print(f"   {CONFIG['episodes']} episodes")
print(f"   Exploration: {CONFIG['exploration_start']*100:.0f}% → {CONFIG['exploration_end']*100:.0f}%")
print(f"   Buy boost: {CONFIG['buy_boost']}x")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading 2 years of data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

print(f"\n✅ Loaded {len(data_dict)} tickers")

In [ ]:
# === CELL 4: FEATURE ENGINE (Fixed NaN issue!) ===
def compute_features(df):
    """Compute all features with proper NaN handling"""
    df = df.copy()
    close = df['Close'].astype(float)
    high = df['High'].astype(float)
    low = df['Low'].astype(float)
    volume = df['Volume'].astype(float)
    
    # === RETURNS ===
    for p in [1, 2, 3, 5, 8, 10, 13, 21, 34]:
        df[f'ret_{p}d'] = close.pct_change(p) * 100
    
    # === VOLATILITY ===
    for p in [5, 10, 21]:
        df[f'vol_{p}d'] = df['ret_1d'].rolling(p).std()
    
    # === EMAs (Your ribbon!) ===
    for p in [5, 8, 13, 21, 34, 55, 89]:
        df[f'ema_{p}'] = close.ewm(span=p).mean()
        df[f'price_vs_ema_{p}'] = (close / df[f'ema_{p}'] - 1) * 100
    
    # EMA slopes (rising/falling)
    for p in [8, 13, 21]:
        df[f'ema_{p}_rising'] = (df[f'ema_{p}'] > df[f'ema_{p}'].shift(3)).astype(float)
    
    # Ribbon analysis
    ribbon_cols = ['ema_8', 'ema_13', 'ema_21', 'ema_34', 'ema_55']
    df['ribbon_min'] = df[ribbon_cols].min(axis=1)
    df['ribbon_max'] = df[ribbon_cols].max(axis=1)
    df['ribbon_range'] = (df['ribbon_max'] - df['ribbon_min']) / (df['ribbon_min'] + 1e-10) * 100
    df['ribbon_bullish'] = ((df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])).astype(float)
    df['ribbon_bearish'] = ((df['ema_8'] < df['ema_13']) & (df['ema_13'] < df['ema_21'])).astype(float)
    df['ribbon_tight'] = (df['ribbon_range'] < 5).astype(float)
    
    # === RSI ===
    for p in [9, 14, 21]:
        delta = close.diff()
        gain = delta.where(delta > 0, 0).rolling(p).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(p).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{p}'] = 100 - (100 / (1 + rs))
    
    df['rsi_slope'] = df['rsi_14'] - df['rsi_14'].shift(5)
    
    # === MACD ===
    for fast, slow in [(8, 21), (12, 26)]:
        ema_fast = close.ewm(span=fast).mean()
        ema_slow = close.ewm(span=slow).mean()
        df[f'macd_{fast}_{slow}'] = ema_fast - ema_slow
        df[f'macd_signal_{fast}_{slow}'] = df[f'macd_{fast}_{slow}'].ewm(span=9).mean()
        df[f'macd_hist_{fast}_{slow}'] = df[f'macd_{fast}_{slow}'] - df[f'macd_signal_{fast}_{slow}']
        df[f'macd_rising_{fast}_{slow}'] = (df[f'macd_hist_{fast}_{slow}'] > df[f'macd_hist_{fast}_{slow}'].shift(1)).astype(float)
    
    # === BOLLINGER ===
    for p in [20]:
        mid = close.rolling(p).mean()
        std = close.rolling(p).std()
        df['bb_upper'] = mid + 2 * std
        df['bb_lower'] = mid - 2 * std
        df['bb_pct'] = (close - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (mid + 1e-10) * 100
    
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean() * 0.75).astype(float)
    
    # === VOLUME ===
    df['vol_sma_20'] = volume.rolling(20).mean()
    df['vol_ratio'] = volume / (df['vol_sma_20'] + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2.0).astype(float)
    
    # === MOMENTUM ===
    df['mom_5d'] = close.pct_change(5) * 100
    df['mom_10d'] = close.pct_change(10) * 100
    df['mom_accel'] = df['mom_5d'] - df['mom_5d'].shift(1)
    
    # === BOUNCE (Your pattern!) ===
    df['low_5d'] = low.rolling(5).min()
    df['bounce'] = (close / (df['low_5d'] + 1e-10) - 1) * 100
    df['bounce_signal'] = ((df['bounce'] > 3) & (df['ema_8_rising'] > 0)).astype(float)
    
    # === TREND ===
    df['trend_5d'] = np.sign(df['ret_5d'])
    df['trend_10d'] = np.sign(df['ret_10d'])
    df['trend_21d'] = np.sign(df['ret_21d'])
    df['trend_align'] = (df['trend_5d'] + df['trend_10d'] + df['trend_21d']) / 3
    
    # === ATR ===
    tr = pd.concat([
        high - low,
        (high - close.shift()).abs(),
        (low - close.shift()).abs()
    ], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_pct'] = df['atr_14'] / (close + 1e-10) * 100
    
    # === SUPPORT/RESISTANCE ===
    df['resist_20d'] = high.rolling(20).max()
    df['support_20d'] = low.rolling(20).min()
    df['near_resist'] = ((df['resist_20d'] - close) / (close + 1e-10) < 0.02).astype(float)
    df['near_support'] = ((close - df['support_20d']) / (close + 1e-10) < 0.02).astype(float)
    
    # === CLEAN UP (Critical for avoiding NaN errors!) ===
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill()
    df = df.fillna(0)
    
    return df

print("🧠 Computing features...")
features = {}
for ticker, df in data_dict.items():
    features[ticker] = compute_features(df)

# Count features
sample_df = features[list(features.keys())[0]]
feature_cols = [c for c in sample_df.columns if c not in ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']]
print(f"✅ {len(feature_cols)} features computed for {len(features)} tickers")

In [ ]:
# === CELL 5: TRADING ENGINE ===
class SwingTrader:
    def __init__(self, balance=100000):
        self.initial = balance
        self.balance = balance
        self.positions = {}
        self.trades = []
        self.history = [balance]
    
    def reset(self):
        self.balance = self.initial
        self.positions = {}
        self.trades = []
        self.history = [self.initial]
    
    def portfolio_value(self, prices):
        val = self.balance
        for t, p in self.positions.items():
            if t in prices:
                val += p['shares'] * prices[t]
        return val
    
    def buy(self, ticker, price, pct=0.25):
        if ticker in self.positions:
            return False
        shares = int(self.balance * pct / price)
        if shares > 0 and shares * price < self.balance * 0.97:
            self.balance -= shares * price
            self.positions[ticker] = {
                'shares': shares,
                'entry': price,
                'max': price,
                'days': 0
            }
            return True
        return False
    
    def sell(self, ticker, price):
        if ticker not in self.positions:
            return None
        pos = self.positions[ticker]
        pnl = (price / pos['entry'] - 1)
        self.balance += pos['shares'] * price
        trade = {
            'ticker': ticker,
            'pnl': pnl,
            'days': pos['days'],
            'entry': pos['entry'],
            'exit': price
        }
        self.trades.append(trade)
        del self.positions[ticker]
        return trade

print("✅ Trading engine ready")

In [ ]:
# === CELL 6: UNLEASHED TRAINING LOOP ===
def run_episode(trader, exploration=0.5):
    """Run one episode with MAXIMUM freedom to learn"""
    trader.reset()
    tickers = list(features.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(60, min_len - 1):
        prices = {}
        
        # Update positions
        for t in list(trader.positions.keys()):
            if t in data_dict and day < len(data_dict[t]):
                price = float(data_dict[t]['Close'].iloc[day])
                prices[t] = price
                trader.positions[t]['days'] += 1
                if price > trader.positions[t]['max']:
                    trader.positions[t]['max'] = price
        
        # Process each ticker
        for ticker in tickers:
            if day >= len(features[ticker]):
                continue
            
            df = features[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            # Get indicators
            rsi = float(df['rsi_14'].iloc[day])
            mom = float(df['mom_5d'].iloc[day])
            ribbon_bull = float(df['ribbon_bullish'].iloc[day])
            ribbon_tight = float(df['ribbon_tight'].iloc[day])
            macd_up = float(df['macd_rising_12_26'].iloc[day])
            bb_squeeze = float(df['bb_squeeze'].iloc[day])
            bounce = float(df['bounce_signal'].iloc[day])
            trend = float(df['trend_align'].iloc[day])
            vol_spike = float(df['vol_spike'].iloc[day])
            ema8_up = float(df['ema_8_rising'].iloc[day])
            ret_21d = float(df['ret_21d'].iloc[day])
            
            # === SELL LOGIC ===
            if ticker in trader.positions:
                pos = trader.positions[ticker]
                pnl = (price / pos['entry'] - 1) * 100
                from_max = (price / pos['max'] - 1) * 100
                
                sell = False
                
                # Take profits
                if pnl >= CONFIG['profit_target_max']:
                    sell = True
                elif pnl >= CONFIG['profit_target_2'] and rsi > 75:
                    sell = True
                elif pnl >= CONFIG['profit_target_1'] and ribbon_bull < 1:
                    sell = True
                
                # Stop loss
                if pnl <= CONFIG['stop_loss']:
                    sell = True
                
                # Trailing stop
                if pnl > 20 and from_max < -CONFIG['trailing_stop']:
                    sell = True
                
                # Time exit
                if pos['days'] > CONFIG['max_hold_days'] and pnl > 0:
                    sell = True
                
                # Exploration sells
                if np.random.random() < exploration * 0.05:
                    sell = np.random.random() < 0.15
                
                if sell:
                    trader.sell(ticker, price)
            
            # === BUY LOGIC (UNLEASHED!) ===
            else:
                if len(trader.positions) >= CONFIG['max_positions']:
                    continue
                
                pv = trader.portfolio_value(prices)
                if trader.balance / pv < CONFIG['min_cash_reserve'] / 100:
                    continue
                
                # Count signals (VERY loose criteria!)
                signals = 0
                
                if rsi < CONFIG['dip_buy_rsi']:
                    signals += 1
                if ret_21d < CONFIG['dip_buy_drawdown']:
                    signals += 1
                if bounce > 0:
                    signals += 1
                if mom > CONFIG['momentum_threshold']:
                    signals += 1
                if ribbon_bull > 0:
                    signals += 1
                if macd_up > 0:
                    signals += 1
                if trend > 0:
                    signals += 1
                if vol_spike > 0 and mom > 0:
                    signals += 1
                if ema8_up > 0:
                    signals += 1
                if bb_squeeze > 0 and ribbon_bull > 0:
                    signals += 2
                if ribbon_tight > 0:
                    signals += 1
                if rsi < 35 and macd_up > 0:
                    signals += 3  # Strong dip buy
                if mom > 10 and rsi < 65:
                    signals += 2  # Momentum
                
                # Buy if ANY signal
                buy = signals >= 1
                
                # EXPLORATION BUYS (learn by doing!)
                if np.random.random() < exploration * CONFIG['buy_boost']:
                    buy = True
                
                if buy:
                    trader.buy(ticker, price, CONFIG['max_position_pct']/100)
        
        trader.history.append(trader.portfolio_value(prices))
    
    # Liquidate
    for t in list(trader.positions.keys()):
        if t in data_dict:
            trader.sell(t, float(data_dict[t]['Close'].iloc[-1]))
    
    return trader.balance, trader.trades

print("✅ Training loop ready")

In [ ]:
# === CELL 7: RUN TRAINING ===
trader = SwingTrader(CONFIG['initial_balance'])

best_return = -999
best_trades = []
all_results = []

print("=" * 65)
print("🔥 UNLEASHED TRAINING - Let it Learn by Making Mistakes!")
print("=" * 65)
print(f"Episodes: {CONFIG['episodes']}")
print(f"Tickers: {len(features)}")
print(f"Exploration: {CONFIG['exploration_start']*100:.0f}% → {CONFIG['exploration_end']*100:.0f}%")
print("-" * 65)

start = time.time()

for ep in range(CONFIG['episodes']):
    # Exploration decay
    progress = ep / CONFIG['episodes']
    exploration = CONFIG['exploration_start'] - progress * (CONFIG['exploration_start'] - CONFIG['exploration_end'])
    
    # Run episode
    final_val, trades = run_episode(trader, exploration)
    ret = (final_val / CONFIG['initial_balance'] - 1) * 100
    
    # Track best
    if ret > best_return:
        best_return = ret
        best_trades = trades.copy()
    
    all_results.append({'ep': ep, 'return': ret, 'trades': len(trades)})
    
    # Progress every 50 episodes
    if ep % 50 == 0:
        wins = len([t for t in trades if t['pnl'] > 0])
        wr = wins / max(len(trades), 1) * 100
        elapsed = time.time() - start
        print(f"Ep {ep:4d} | Ret: {ret:+7.1f}% | Best: {best_return:+7.1f}% | "
              f"Trades: {len(trades):3d} | WR: {wr:4.0f}% | Exp: {exploration*100:4.0f}% | {elapsed:5.0f}s")

print("-" * 65)
print(f"\n🏆 TRAINING COMPLETE in {time.time()-start:.0f}s")
print(f"   Best Return: {best_return:+.1f}%")
print(f"   Best Trades: {len(best_trades)}")

In [ ]:
# === CELL 8: ANALYZE RESULTS ===
winners = [t for t in best_trades if t['pnl'] > 0]
losers = [t for t in best_trades if t['pnl'] <= 0]

print("=" * 65)
print("📊 TRAINING RESULTS")
print("=" * 65)
print(f"Total Trades: {len(best_trades)}")
print(f"Winners: {len(winners)} | Losers: {len(losers)}")
print(f"Win Rate: {len(winners)/max(len(best_trades),1)*100:.1f}%")

if winners:
    print(f"Avg Winner: {np.mean([t['pnl'] for t in winners])*100:+.1f}%")
    print(f"Best Win: {max([t['pnl'] for t in winners])*100:+.1f}%")
if losers:
    print(f"Avg Loser: {np.mean([t['pnl'] for t in losers])*100:+.1f}%")
    print(f"Worst Loss: {min([t['pnl'] for t in losers])*100:+.1f}%")

print(f"\n🏆 TOP 15 WINNING TRADES:")
for t in sorted(winners, key=lambda x: x['pnl'], reverse=True)[:15]:
    print(f"   {t['ticker']:6s}: {t['pnl']*100:+6.1f}% in {t['days']:2d} days")

# By ticker
print(f"\n📈 PERFORMANCE BY TICKER:")
ticker_pnl = {}
for t in best_trades:
    ticker_pnl.setdefault(t['ticker'], []).append(t['pnl'])

for tk in sorted(ticker_pnl.keys(), key=lambda x: sum(ticker_pnl[x]), reverse=True)[:15]:
    pnls = ticker_pnl[tk]
    wins = len([p for p in pnls if p > 0])
    print(f"   {tk:6s}: {sum(pnls)*100:+7.1f}% total | {len(pnls):2d} trades | {wins/len(pnls)*100:4.0f}% WR")

In [ ]:
# === CELL 9: TODAY'S SIGNALS ===
print("=" * 65)
print("🎯 TODAY'S BUY SIGNALS")
print("=" * 65)

signals = []
for ticker in features.keys():
    df = features[ticker]
    idx = -1
    
    price = float(data_dict[ticker]['Close'].iloc[idx])
    rsi = float(df['rsi_14'].iloc[idx])
    mom = float(df['mom_5d'].iloc[idx])
    ribbon_bull = float(df['ribbon_bullish'].iloc[idx])
    ribbon_tight = float(df['ribbon_tight'].iloc[idx])
    macd_up = float(df['macd_rising_12_26'].iloc[idx])
    bb_squeeze = float(df['bb_squeeze'].iloc[idx])
    bounce = float(df['bounce_signal'].iloc[idx])
    trend = float(df['trend_align'].iloc[idx])
    ret_21d = float(df['ret_21d'].iloc[idx])
    
    sigs = []
    score = 0
    
    if rsi < 40:
        sigs.append('RSI_LOW')
        score += 2
    elif rsi < 50:
        sigs.append('RSI_MID')
        score += 1
    
    if mom > 10:
        sigs.append('MOMENTUM🚀')
        score += 3
    elif mom > 5:
        sigs.append('MOM_UP')
        score += 1
    
    if ribbon_bull > 0:
        sigs.append('RIBBON_BULL📈')
        score += 2
    
    if ribbon_tight > 0 and ribbon_bull > 0:
        sigs.append('SQUEEZE💥')
        score += 2
    
    if macd_up > 0:
        sigs.append('MACD_UP')
        score += 1
    
    if bounce > 0:
        sigs.append('BOUNCE⬆️')
        score += 2
    
    if bb_squeeze > 0:
        sigs.append('BB_SQUEEZE')
        score += 1
    
    if trend > 0.5:
        sigs.append('TREND_UP')
        score += 1
    
    if ret_21d < -5:
        sigs.append('DIP🎯')
        score += 2
    
    if sigs:
        signals.append({
            'ticker': ticker,
            'price': price,
            'score': score,
            'signals': sigs,
            'rsi': rsi,
            'mom': mom
        })

signals.sort(key=lambda x: x['score'], reverse=True)

print(f"\n🔥 {len(signals)} STOCKS WITH SIGNALS:\n")
for s in signals:
    stars = '⭐' * min(s['score'] // 2, 5)
    print(f"{stars} {s['ticker']:6s} ${s['price']:>8.2f} | RSI:{s['rsi']:4.0f} | Mom:{s['mom']:+5.1f}%")
    print(f"         {' + '.join(s['signals'][:4])}")
    print()

In [ ]:
# === CELL 10: SAVE RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'config': CONFIG,
    'performance': {
        'best_return': best_return,
        'total_trades': len(best_trades),
        'win_rate': len(winners) / max(len(best_trades), 1) * 100,
        'avg_winner': np.mean([t['pnl'] for t in winners]) * 100 if winners else 0,
        'avg_loser': np.mean([t['pnl'] for t in losers]) * 100 if losers else 0,
    },
    'top_trades': sorted(winners, key=lambda x: x['pnl'], reverse=True)[:30],
    'ticker_performance': {
        tk: {'total': sum(pnls)*100, 'trades': len(pnls)}
        for tk, pnls in ticker_pnl.items()
    },
    'todays_signals': signals[:20]
}

with open('unleashed_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to unleashed_results.json")

# Download
try:
    from google.colab import files
    files.download('unleashed_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")